In [1]:
import docker
import uuid
import paramiko
import os
from timeit import default_timer as timer
from dataclasses import dataclass
import re
import requests
import statistics
import pandas as pd
import numpy as np

In [5]:
def run_in_master(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("namenode", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"cd /app/ && . /env_var_path.sh && {command}")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def run_in_hive(command):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect("hive-server", username="root", password="pass")
    ssh_stdin, ssh_stdout, ssh_stderr = ssh.exec_command(f"bash -c '. /env_var_path.sh && {command}'")
    return (ssh_stdout.readlines(), ssh_stderr.readlines())

def get_hive_time_from_results(res):
    all = []
    for line in res[1]:
        m = re.search('Time taken: (.*) \w+', line)
        if m != None and m.group(1) != '':
            all.append(float(m.group(1)))
    return sum(all)

In [8]:
run_in_hive("hive -f /data/master_volume/hive_scripts/covid_table.hql")
covid_load_res = run_in_hive("hive -f /data/master_volume/hive_scripts/covid_load_to_hive.hql")
covid_res = run_in_hive("hive -f /data/master_volume/hive_scripts/covid_data.hql")

covid_load_time = get_hive_time_from_results(covid_load_res)
covid_time = get_hive_time_from_results(covid_res)

In [9]:
covid_load_time + covid_time

50.10600000000001

In [ ]:
run_in_master(f"hdfs dfs -cat /user/hive/warehouse/coviddb.db/calculated/000000_0")

In [13]:
steam_res = run_in_hive("hive -f /data/master_volume/hive_scripts/steam_01_combine.hql")
steam_time = get_hive_time_from_results(steam_res)

In [14]:
steam_time

85.789

In [15]:
run_in_hive("hive -f /data/master_volume/hive_scripts/steam_01_combine_show.hql")

(['570\tDota 2\t1456697\t291673\t100,000,000 .. 200,000,000\t612293\tJul 9, 2013\n',
  '730\tCounter-Strike: Global Offensive\t5680980\t756795\t50,000,000 .. 100,000,000\t679488\tAug 21, 2012\n',
  '578080\tPUBG: BATTLEGROUNDS\t1139437\t888720\t50,000,000 .. 100,000,000\t279181\tDec 21, 2017\n',
  '1063730\tNew World\t153807\t73148\t50,000,000 .. 100,000,000\t20403\tSep 28, 2021\n',
  '440\tTeam Fortress 2\t813122\t56073\t50,000,000 .. 100,000,000\t82571\tOct 10, 2007\n',
  '304930\tUnturned\t438411\t40925\t20,000,000 .. 50,000,000\t50423\tJul 7, 2017\n',
  '271590\tGrand Theft Auto V\t1144201\t208030\t20,000,000 .. 50,000,000\t109264\tApr 13, 2015\n',
  '550\tLeft 4 Dead 2\t600580\t15752\t20,000,000 .. 50,000,000\t20819\tNov 16, 2009\n',
  '230410\tWarframe\t437656\t47546\t20,000,000 .. 50,000,000\t50453\tMar 25, 2013\n',
  '105600\tTerraria\t941388\t20346\t20,000,000 .. 50,000,000\t39371\tMay 16, 2011\n'],
 ['SLF4J: Class path contains multiple SLF4J bindings.\n',
  'SLF4J: Found bin